## Agent Framework Workflows Orchestrations
Traditional single-agent systems are limited in their ability to handle complex, multi-faceted tasks. 

Multi-agent orchestration addresses this limitation by allowing you to:

- Assign distinct skills, responsibilities, or perspectives to each agent.
- Combine outputs from multiple agents to improve decision-making and accuracy.
- Coordinate steps in a workflow so each agent’s work builds on the last.
- Dynamically route control between agents based on context or rules.

**Orchestrations** are pre-built workflow patterns that allow developers to quickly create complex workflows by simply plugging in their own AI agents. Instead of manually wiring every edge, you can use these patterns to define how agents interact.

Because all patterns share the same core interface, you can easily experiment with different orchestration strategies without rewriting agent logic or learning new APIs. The SDK abstracts the complexity of agent communication, coordination, and result aggregation so you can focus on designing workflows that deliver results.

Agent Framework provides five orchestration patterns:

1. **Sequential** - Agents execute one after another in a defined order, ideal for linear processing
2. **Concurrent** - Broadcast the same task to multiple agents at once and collect results independently.
3. **Handoff** - Control is dynamically transferred from one agent to another based on context or role.
4. **Group chat** - Coordinate a shared conversation among multiple agents managed by a chat manager that decides who speaks next and when to stop.
5. **Magentic** - Manager-driven approach where agents dynamically pull tasks based on the evolving context, task progress, and agent capabilities.

Refer to the [Agent Framework documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/workflows/orchestrations/overview) to learn more about these orchestration types and check out the [Agent Framework repo](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/workflows/orchestration) to see more advanced orchestration types in action.

In this notebook, we'll cover **Sequential**, **Concurrent**, and **Group Chat** orchestrations in detail. Handoff and Magentic are introduced in the exercise section with links to documentation and samples.

### Setup

First, let's validate our environment and create the Azure OpenAI client.

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

from workshop_utils import validate_env

validate_env()

In [ ]:
import os
from agent_framework.azure import AzureOpenAIChatClient

chat_client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
)

print("Client created successfully!")

## Sequential Orchestration

In sequential orchestration, agents are organized in a pipeline. Each agent processes the task in turn, passing its output to the next agent in the sequence. This is ideal for workflows where each step builds upon the previous one, such as document review, data processing pipelines, or multi-stage reasoning.

The `SequentialBuilder` class creates a pipeline where agents process tasks in order. Each agent sees the full conversation history and adds their response.

In [ ]:
from agent_framework import SequentialBuilder, ChatMessage, Role, WorkflowOutputEvent

# 1) Create agents from chat_client:
meal_planner = chat_client.as_agent(
    instructions=(
        "You are a meal planner. Suggest a simple, healthy weekly meal plan for breakfast, lunch, and dinner."
    ),
    name="meal_planner",
)

budget_estimator = chat_client.as_agent(
    instructions=(
        "You are a budget estimator. Calculate an approximate total cost for the 7-day meal plan provided by the previous assistant."
    ),
    name="budget_estimator",
)

# 2) Build sequential workflow: meal_planner -> budget_estimator
workflow = SequentialBuilder().participants([meal_planner, budget_estimator]).build()

# 3) Run and capture the final output event
output_evt: WorkflowOutputEvent | None = None
async for event in workflow.run_stream("Create a short weekly meal plan for one person."):
    if isinstance(event, WorkflowOutputEvent):
        output_evt = event

# 4) Print final conversation from the output event
if output_evt:
    print("===== Final Conversation =====")
    messages: list[ChatMessage] = output_evt.data
    for i, msg in enumerate(messages, start=1):
        name = msg.author_name or ("assistant" if msg.role == Role.ASSISTANT else "user")
        print(f"{'-' * 60}\n{i:02d} [{name}]\n{msg.text}")

## Concurrent Orchestration
Concurrent orchestration enables multiple agents to work on the same task in parallel. Each agent processes the input independently, and their results are collected and aggregated. This approach is well-suited for scenarios where diverse perspectives or solutions are valuable. 
The `ConcurrentBuilder` class allows you to construct a workflow that runs multiple agents in parallel.

- You pass a list of agents as participants.
- The builder automatically orchestrates their execution concurrently.
- Results are returned as a collection of outputs from all agents.

The following example demonstrates how to:

1. Build a concurrent workflow using `ConcurrentBuilder`
2. Fan-out to multiple agents in parallel
3. Fan-in and aggregate their outputs

In [ ]:
from agent_framework import ConcurrentBuilder, ChatMessage, WorkflowOutputEvent

# 1) Create three domain agents using AzureOpenAIChatClient
researcher = chat_client.as_agent(
    instructions=(
        "You're an expert market and product researcher. Given a prompt, provide concise, factual insights,"
        " opportunities, and risks."
    ),
    name="researcher",
)

marketer = chat_client.as_agent(
    instructions=(
        "You're a creative marketing strategist. Craft compelling value propositions and target messaging"
        " aligned to the prompt."
    ),
    name="marketer",
)

legal = chat_client.as_agent(
    instructions=(
        "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, and policy concerns"
        " based on the prompt."
    ),
    name="legal",
)

# 2) Build a concurrent workflow
workflow = ConcurrentBuilder().participants([researcher, marketer, legal]).build()

# 3) Run with streaming and capture the final output event
prompt = "We are launching a new budget-friendly electric bike for urban commuters."
output_evt: WorkflowOutputEvent | None = None
async for event in workflow.run_stream(prompt):
    if isinstance(event, WorkflowOutputEvent):
        output_evt = event

# 4) Print outputs from each agent
if output_evt:
    print("===== Concurrent Agent Outputs =====")
    messages: list[ChatMessage] = output_evt.data
    for msg in messages:
        name = msg.author_name or "user"
        print(f"{'-' * 60}\n[{name}]:\n{msg.text}\n")

## Group Chat Orchestration

**Group chat orchestration** models a collaborative conversation among multiple AI agents and optionally a human participant. A central chat manager controls the flow and decides which agent responds next or when to request human input.

Group chat is ideal for iterative refinement, debate, and multi-perspective problem solving where agents share the same conversation context and can build on each other's outputs.

**Core parts of Group chat orchestration:**
1. **Agents** - Individual ChatAgents with distinct roles (e.g., Planner, Engineer, Reviewer). Each agent can reason independently but collaborates through the orchestrator.

2. **`GroupChatBuilder()`** - Fluent API to assemble a group chat with participants, speaker selection strategy, and termination conditions.

3. **`GroupChatState`** - State object passed to the speaker selector with these attributes:
    - `participants`: dict mapping agent names to descriptions
    - `conversation`: list of ChatMessage objects (full history)
    - `current_round`: number of selection rounds so far

4. **Selection Strategy** - The policy that decides who speaks next. Options include:
    - **Custom function**: Use `.with_select_speaker_func(func)` for deterministic selection
    - **Agent-based orchestrator**: Use `.with_agent_orchestrator(agent)` for AI-managed selection

5. **Termination Condition** - Use `.with_termination_condition(func)` to define when the conversation should end.

### Example - Group Chat with Custom Speaker Selection

In the following example we'll implement a three agent group chat with a simple round-robin speaker selector.

Key Objectives:
- Implement a deterministic speaker selection function using `GroupChatState`
- Use `.with_select_speaker_func()` to register the selector
- Use `.with_termination_condition()` to end after a set number of turns
- Stream events for real-time observability

First, let's create agents with detailed instructions:

In [ ]:
from agent_framework import agent_middleware

@agent_middleware 
async def logging_agent_middleware(context, next):  
    """Agent middleware with decorator - types are inferred."""
    print(f"[{context.agent.name} Agent] INVOKING")
    await next(context)   
    print(f"[{context.agent.name} Agent] FINISHED")

writer = chat_client.as_agent(
    name="Writer",
    instructions="""
    You are a creative writer crafting engaging content.

    Role:
    1. Generate original, compelling narratives.
    2. Apply storytelling techniques for engagement.
    3. Incorporate feedback from editor/researcher.
    4. Revise to address issues while preserving core message.

    Feedback:
    - Accept constructive criticism, Explain creative choices when needed, Keep responses concise, imaginative, and engaging.""",
    middleware=[logging_agent_middleware]
)

editor = chat_client.as_agent(
    name="Editor",
    instructions="""
    You are an editor focused on clarity and quality.

    Role:
    1. Ensure clarity, coherence, and flow.
    2. Fix grammar, structure, and style.
    3. Suggest improvements for readability and impact.

    Feedback:
    - Be specific and constructive, Address sentence-level and structural issues, Respect writer's voice while improving content.
    Keep responses concise and clear.""",
    middleware=[logging_agent_middleware]
)


researcher = chat_client.as_agent(
    name="Researcher",
    instructions="""
    You provide accurate, relevant research.

    Role:
    1. Gather credible, up-to-date info.
    2. Summarize findings clearly.
    3. Highlight key insights and fill gaps.
    4. Ensure factual accuracy and alignment with goals.

    Responses:
    - Base on verifiable facts, Use bullet points or short paragraphs, Suggest sources when useful.
    Keep responses concise, relevant, and actionable.""",
    middleware=[logging_agent_middleware]
)


In [ ]:
import logging

from agent_framework import (
    ChatMessage, 
    GroupChatBuilder,
    GroupChatState,
    WorkflowOutputEvent, 
    AgentRunUpdateEvent
)

logging.basicConfig(level=logging.INFO)

def select_next_speaker(state: GroupChatState) -> str:
    """Round-robin speaker selection based on conversation state."""
    participants = list(state.participants.keys())
    current_round = state.current_round
    
    # Cycle through participants in preferred order
    preferred = ["Researcher", "Writer", "Editor"]
    order = [p for p in preferred if p in participants]
    
    # Return the next speaker based on current round
    return order[current_round % len(order)]


workflow = (
    GroupChatBuilder()
    .with_select_speaker_func(select_next_speaker, orchestrator_name="Orchestrator")
    .participants([researcher, writer, editor])
    .with_termination_condition(lambda conversation: len(conversation) >= 6)  # Stop after 6 messages
    .build()
)

task = "What are the key benefits of using async/await in Python?"

print("\nStarting Group Chat with Round-Robin Speaker Selection...\n")
print(f"TASK: {task}\n")
print("=" * 80)

async for event in workflow.run_stream(
    ChatMessage(role="user", text=task)
):
    if isinstance(event, AgentRunUpdateEvent):
        print(f"{event.data}", end="", flush=True)
    elif isinstance(event, WorkflowOutputEvent):
        print(f"\n\nWorkflow output received.")

print("\nWorkflow completed.")

 **Note:**
In the output, you might notice that middleware logs FINISHED before the agent-generated content appears. 

In streaming mode, the agent returns an async generator, so the FINISHED log simply means the generator was created and handed back, not that all tokens have been printed. The actual text follows via `AgentRunUpdateEvent` as tokens stream progressively.

### Choosing an Orchestration

When selecting an orchestration pattern, consider these guidelines:

| Pattern | Best For |
|---------|----------|
| **Sequential** | Pipelines where each step builds on the previous (draft → review → polish) |
| **Concurrent** | Independent parallel work with aggregated results (multi-perspective analysis) |
| **Group Chat** | Iterative collaboration, maker-checker loops, human-in-the-loop |
| **Handoff** | Dynamic routing when expertise requirements emerge during processing |
| **Magentic** | Manager-driven task distribution based on evolving context |

### Exercise: Build a Product Launch Review System

**Scenario**: Your company is launching a new AI-powered fitness app. Before launch, you need multiple teams to review and provide feedback.

| Agent | Role | Focus |
|-------|------|-------|
| `product_manager` | Product Strategy | Feature completeness, market fit, user value |
| `security_reviewer` | Security & Privacy | Data handling, GDPR compliance, vulnerabilities |
| `ux_designer` | User Experience | Usability, accessibility, design consistency |

**Your Task**: Choose an orchestration pattern and implement the review system.

**Hints**:
- Consider: Should reviews happen in sequence (each building on the last) or in parallel (independent perspectives)?
- For parallel reviews, use `ConcurrentBuilder`
- For sequential reviews with synthesis, use `SequentialBuilder`
- For iterative discussion, use `GroupChatBuilder`

In [ ]:
# TODO: Create three agents for the product launch review
product_manager = chat_client.as_agent(
    name="product_manager",
    instructions="""You are a product manager reviewing a product launch.
    Focus on: feature completeness, market fit, and user value proposition.
    Provide specific, actionable feedback.""",
)

security_reviewer = chat_client.as_agent(
    name="security_reviewer",
    instructions="""You are a security and privacy reviewer.
    Focus on: data handling practices, GDPR/privacy compliance, potential vulnerabilities.
    Flag any concerns with severity ratings.""",
)

ux_designer = chat_client.as_agent(
    name="ux_designer",
    instructions="""You are a UX designer reviewing a product.
    Focus on: usability, accessibility, design consistency, user journey.
    Suggest specific improvements.""",
)

# TODO: Choose and build your orchestration
# Option 1: ConcurrentBuilder for parallel independent reviews
# Option 2: SequentialBuilder for reviews that build on each other
# Option 3: GroupChatBuilder for iterative discussion

# workflow = ...

# TODO: Run the workflow with this prompt
prompt = """Review our new AI fitness app 'FitAI' for launch readiness:
- Tracks workouts using phone sensors
- Uses AI to generate personalized workout plans
- Stores user health data in the cloud
- Integrates with social media for sharing achievements
- Subscription model: $9.99/month"""

# result = await workflow.run(prompt)
# print(result)

<details>
<summary><strong>Click to reveal solution</strong></summary>

```python
from agent_framework import ConcurrentBuilder, ChatMessage, WorkflowOutputEvent

# Create agents
product_manager = chat_client.as_agent(
    name="product_manager",
    instructions="""You are a product manager reviewing a product launch.
    Focus on: feature completeness, market fit, and user value proposition.
    Provide specific, actionable feedback.""",
)

security_reviewer = chat_client.as_agent(
    name="security_reviewer",
    instructions="""You are a security and privacy reviewer.
    Focus on: data handling practices, GDPR/privacy compliance, potential vulnerabilities.
    Flag any concerns with severity ratings.""",
)

ux_designer = chat_client.as_agent(
    name="ux_designer",
    instructions="""You are a UX designer reviewing a product.
    Focus on: usability, accessibility, design consistency, user journey.
    Suggest specific improvements.""",
)

# Concurrent is a good choice here - each reviewer provides independent perspective
workflow = ConcurrentBuilder().participants([product_manager, security_reviewer, ux_designer]).build()

prompt = """Review our new AI fitness app 'FitAI' for launch readiness:
- Tracks workouts using phone sensors
- Uses AI to generate personalized workout plans
- Stores user health data in the cloud
- Integrates with social media for sharing achievements
- Subscription model: $9.99/month"""

# Run with streaming and capture output
output_evt: WorkflowOutputEvent | None = None
async for event in workflow.run_stream(prompt):
    if isinstance(event, WorkflowOutputEvent):
        output_evt = event

# Print each reviewer's feedback
if output_evt:
    messages: list[ChatMessage] = output_evt.data
    for msg in messages:
        if msg.author_name:
            print(f"\n{'='*60}")
            print(f"[{msg.author_name}]")
            print(f"{'='*60}")
            print(msg.text)
```

</details>

**Bonus Challenge**: Extend your solution to add a `synthesizer` agent that takes all the reviews and creates a consolidated launch readiness report. Consider using Sequential orchestration to chain the concurrent reviews with the synthesis step.

---
## Summary & Recap

In this notebook, you learned about pre-built orchestration patterns in the Agent Framework:

### Key Concepts

| Orchestration | Description |
|---------------|-------------|
| **Sequential** | Agents execute one after another in a pipeline, each building on the previous output |
| **Concurrent** | Multiple agents work on the same task in parallel, results are aggregated |
| **Group Chat** | Collaborative conversation with a chat manager controlling speaker selection |
| **Handoff** | Dynamic control transfer between agents based on context |
| **Magentic** | Manager-driven approach where agents pull tasks based on evolving context |

### What You Built

1. **Sequential Pipeline** - Meal planner and budget estimator working in sequence
2. **Concurrent Workflow** - Researcher, marketer, and legal agents processing in parallel
3. **Group Chat** - Writer, editor, and researcher with custom speaker selection

### Key Patterns

```python
# Sequential orchestration
workflow = SequentialBuilder().participants([agent1, agent2]).build()

# Concurrent orchestration
workflow = ConcurrentBuilder().participants([agent1, agent2, agent3]).build()

# Group chat with custom speaker selection
def select_speaker(state: GroupChatState) -> str:
    # Custom logic to determine next speaker
    return next_agent_name

workflow = (
    GroupChatBuilder()
    .with_select_speaker_func(select_speaker, orchestrator_name=\"Orchestrator\")
    .participants([agent1, agent2, agent3])
    .with_termination_condition(lambda conversation: len(conversation) >= 6)
    .build()
)
```

### Chapter 2 Complete!

You now have the skills to build sophisticated multi-agent systems with:
- Custom workflows with executors and edges (02.1)
- Agents as workflow components (02.1)
- Pre-built orchestration patterns (02.2)
- Event-driven observability and streaming (02.1, 02.2)

In **Chapter 3**, you'll learn about **RAG (Retrieval-Augmented Generation)** to give your agents access to external knowledge!